In [3]:
!pip install requests
!pip install pillow
!pip install spaCy


import base64
import os
import requests
import spacy


engine_id = "stable-diffusion-v1-5"
api_host = os.getenv('API_HOST', 'https://api.stability.ai')
api_key = "sk-83pLdqWNDfPpPRfwpTSMSTwmNbrqpv3AfyV5BRXmvUZuR0fQ"
text_prompt= input("Please enter a sentence: ")

if api_key is None:
    raise Exception("Missing Stability API key.")

response = requests.post(
    f"{api_host}/v1/generation/{engine_id}/text-to-image",
    headers={
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {api_key}"
    },
        json={
        "text_prompts": [
            {
                "text": text_prompt
            }
        ],
        "cfg_scale": 7,
        "clip_guidance_preset": "FAST_BLUE",
        "height": 512,
        "width": 512,
        "samples": 1,
        "steps": 30,
    },
)
print(response)
if response.status_code != 200:
    raise Exception("Non-200 response: " + str(response.text))

data = response.json()

for i, image in enumerate(data["artifacts"]):
    with open(f"./out/v1_txt2img_{i}.png", "wb") as f:
        f.write(base64.b64decode(image["base64"]))



nlp = spacy.load("en_core_web_md")
remainingTrials = 3
doc2 = text_prompt
#ask for a sentence
while remainingTrials > 0:
    doc1 = input("Please enter a sentence: ")
    doc1 = nlp(doc1)
    doc2 = nlp(doc2)
    similarity = doc1.similarity(doc2)
    
    percentage = similarity * 100
    if similarity > 0.9:
        print("You win!, the original sentence was: ", doc2)
        break
    else:
        print("Try again, current similarity is ", percentage, "%")
        remainingTrials -= 1
        print("You have ", remainingTrials, " remaining trials")

        if remainingTrials == 0:
            print("You lost, the original sentence was: ", doc2)
            break




<Response [200]>
Try again, current similarity is  81.32192021539164 %
You have  2  remaining trials
You win!, the original sentence was:  A cat on the couch
